<a href="https://colab.research.google.com/github/yev34/Machine-learning/blob/main/%D0%9F%D0%B0%D1%82%D0%B0%D0%BB%D0%B0%D1%88%D0%BA%D0%BE_%D0%BB%D0%B0%D0%B1_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357286 sha256=cc8075d49d9b5c002d403c6e1d83b983d458f82efca2aa038e86ccedc58dbeec
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [11]:
from surprise import Dataset, Reader
from surprise import KNNBasic, SVD
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split

Завантажте датасет для рецензій (ml-100k):

In [12]:


data = Dataset.load_builtin('ml-100k')

Виведіть перші 5 рядків:

In [13]:
df = data.build_full_trainset().build_anti_testset()
print("Перші 5 рецензій:")
for i in range(5):
    print(df[i])

Перші 5 рецензій:
('196', '302', 3.52986)
('196', '377', 3.52986)
('196', '51', 3.52986)
('196', '346', 3.52986)
('196', '474', 3.52986)


Реалізуйте два алгоритми для рекомендаційної системи:

In [14]:
algo1 = KNNBasic()
algo2 = SVD()

Використайте крос-валідацію для підбору параметрів:

In [15]:
results_algo1 = cross_validate(algo1, data, measures=['MAE'], cv=5, verbose=True)
results_algo2 = cross_validate(algo2, data, measures=['MAE'], cv=5, verbose=True)


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.7759  0.7722  0.7763  0.7789  0.7650  0.7737  0.0049  
Fit time          0.32    0.34    0.34    0.34    0.34    0.34    0.01    
Test time         3.37    3.10    4.31    3.15    3.16    3.42    0.45    
Evaluating MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.7420  0.7381  0.7360  0.7393  0.7375  0.7386  0.0020  
Fit time          1.78    2.27    1.43    1.41    1.42    1.66    0.33    
Test time

Оберіть найкращий алгоритм:

In [16]:
mean_mae_algo1 = results_algo1['test_mae'].mean()
mean_mae_algo2 = results_algo2['test_mae'].mean()

print(f"MAE для KNNBasic: {mean_mae_algo1:.4f}")
print(f"MAE для SVD: {mean_mae_algo2:.4f}")

best_algo = algo1 if mean_mae_algo1 < mean_mae_algo2 else algo2
print(f"Найкращий алгоритм: {best_algo}")

trainset = data.build_full_trainset()
best_algo.fit(trainset)

MAE для KNNBasic: 0.7737
MAE для SVD: 0.7386
Найкращий алгоритм: <surprise.prediction_algorithms.matrix_factorization.SVD object at 0x7fc0f806d630>


Вивести рекомендації для конкретного користувача:

In [25]:
user_id = str(67)
items = trainset.all_items()

items_not_rated = [item for item in items if not trainset.knows_item(item)]
print(f"Кількість фільмів, які не оцінені користувачем {user_id}: {len(items_not_rated)}")

if len(items_not_rated) == 0:
    print(f"Користувач {user_id} оцінив усі фільми!")
else:
    predictions = [best_algo.predict(user_id, item) for item in items_not_rated]
    print(f"Кількість прогнозів: {len(predictions)}")

    for prediction in predictions[:5]:
        print(f"Movie ID: {prediction.iid}, Estimated Rating: {prediction.est:.2f}")

    top_10 = sorted(predictions, key=lambda x: x.est, reverse=True)[:10]
    print(f"Кількість фільмів у топ-10: {len(top_10)}")

    print("\nТоп-10 рекомендацій для користувача:")
    for prediction in top_10:
        print(f"Movie ID: {prediction.iid}, Estimated Rating: {prediction.est:.2f}")


Кількість фільмів, які не оцінені користувачем 67: 0
Користувач 67 оцінив усі фільми!


Завдання 2

1. Побудуйте власну рекомендаційну систему:
Ви можете використовувати функцію build_full_trainset для побудови системи з використанням усіх даних і тренування моделі.

In [26]:
trainset = data.build_full_trainset()
algo_svd.fit(trainset)

Отримайте рекомендації для певного користувача:

In [29]:

user_id = '196'

user_ratings = trainset.ur[trainset.to_inner_uid(user_id)]
rated_items = [item_id for (item_id, rating) in user_ratings]


items_not_rated = [item for item in trainset.all_items() if item not in rated_items]

predictions = [algo_svd.predict(user_id, trainset.to_raw_iid(item)) for item in items_not_rated]

top_10 = sorted(predictions, key=lambda x: x.est, reverse=True)[:10]

print("\nТоп-10 рекомендацій для користувача:")
for prediction in top_10:
    print(f"Movie ID: {prediction.iid}, Estimated Rating: {prediction.est:.2f}")



Топ-10 рекомендацій для користувача:
Movie ID: 64, Estimated Rating: 4.65
Movie ID: 318, Estimated Rating: 4.64
Movie ID: 510, Estimated Rating: 4.57
Movie ID: 98, Estimated Rating: 4.56
Movie ID: 483, Estimated Rating: 4.49
Movie ID: 89, Estimated Rating: 4.48
Movie ID: 641, Estimated Rating: 4.47
Movie ID: 661, Estimated Rating: 4.42
Movie ID: 134, Estimated Rating: 4.40
Movie ID: 272, Estimated Rating: 4.40
